Rokas Petrauskas 1gr. 2pogr.

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torch.utils.data as data
from torchvision import transforms, datasets, models
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
from collections import defaultdict
import numpy as np

priskiriam modeli prie GPU, jei unavailable: CPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Modelio atsiuntimas

In [ ]:
model = models.vgg16(pretrained = True).to(device)
model.eval()

Naudojamos klases:
847 tank
895 airplane
471 cannon

Nuotrauku atsiuntimas:
(uzkomentuota, kad leidziant viska nesiustu is naujo, LEIDZIANT NAUJAME IRENGINYJE PAKEISTI dest-dir ir viska atkomentuoti)

In [ ]:
# from openimages.download import download_dataset
# download_dataset (dest_dir = "C:/Users/Rokas/Desktop/univer/gmm/lab1/src", class_labels = ["Tank", "Airplane", "Cannon"], limit = 1000)

paruosiamos nuotraukos 
/ dataloader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
data_dir = 'C:/Users/Rokas/Desktop/univer/gmm/lab1/src'
val_dataset = datasets.ImageFolder(data_dir, transform = transform)

In [ ]:
batch_size = 16
data_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

issaugojam key + value i zodyna

In [ ]:
with open('imagenet_classes.txt', 'r') as f:
    lines = f.read().splitlines()

class_dict = {i+1: lines[i] for i in range(len(lines))}

for key, value in class_dict.items():
    print(key, value)

ground_truths issaugoja tikruju klasiu raktus

In [ ]:
ground_truths = []
class_dict = {'airplane': 895, 'cannon': 471, 'tank': 847}
class_names = list(class_dict.keys())
predictions = defaultdict(lambda: [0]*len(class_names))
dir_path = 'src'
parent_folders = []

for root, dirs, files in os.walk(dir_path):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
            parent_folder = os.path.basename(os.path.dirname(root))
            parent_folders.append(parent_folder)

gt = [class_dict[parent_folder] for parent_folder in parent_folders]
ground_truths.extend(gt)

print(len(ground_truths))
print(ground_truths)



predicted_keys issaugoja predictintu klasiu raktus

In [ ]:
predicted_keys = []
predicted_probs = []
predicted_binary = []
confidence = []

threshold = 0.00001

for images, labels in data_loader:
    # nuotraukos i GPU
    if torch.cuda.is_available():
        images = images.cuda()
    
    # predict
    with torch.no_grad():
        outputs = model(images)
    
    # kiekvieno img predictionas
    probs = torch.nn.functional.softmax(outputs, dim=1)
    _, predicted = torch.max(outputs, 1)
    binary = (probs[:, 0] > threshold).int() ########
    
    # tensor to a list of strings
    predicted_keys_batch = [str(class_id.item()) for class_id in predicted]
    
    # sukraunam i viena list
    predicted_keys.extend(predicted_keys_batch)
    predicted_probs.extend(probs.cpu().numpy())
    predicted_binary.extend(binary.cpu().numpy()) ######

    
print(len(predicted_keys))
print(predicted_keys)
print(predicted_probs)
print(predicted_binary)

softmax -> sigmoid 

In [ ]:
int_predicted_keys = list(map(int, predicted_keys))

In [ ]:
def num_of_samples(source):
    num_files = len([f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f))])
    return(num_files)

calculating correct predictions

In [ ]:
num_of_correct_predictions = 0

for i in range(len(int_predicted_keys)):
    if ground_truths[i] == int_predicted_keys[i]:
        num_of_correct_predictions += 1
    
print(num_of_correct_predictions)
print(len(ground_truths))
print(num_of_correct_predictions/len(ground_truths))

calculating metrics

In [ ]:
klases = [[895, 'airplane', 0], [471, 'cannon', 0], [847, 'tank', 0]]

In [ ]:
# klases[0][2] = num_of_samples('src/airplane/images')
# klases[1][2] = num_of_samples('src/cannon/images')
# klases[2][2] = num_of_samples('src/tank/images')

# num_of_predictions = len(predicted_keys)

# index = 0

# def calculate_met(klase, index):
#     gt = []
#     print(klase)

#     for i in range(index):
#         gt.append(0)

#     for i in range(num_of_predictions):
#         if i <= klase[2]:
#             gt.append(1)
#         else:
#             gt.append(0)

#     index = index + klase[2]

#     print(gt)
#     print(index)
#     return index

# for i in range(3):
#     index = calculate_met(klases[i], index)
    


nepavyko issiaiskint ^ todel ihardcodinau zemiau. might fix if there is still time

In [ ]:
gt_airplane = []
gt_cannon = []
gt_tank = []

num_of_predictions = len(predicted_keys)

for i in range(num_of_samples('src/airplane/images')):
    gt_airplane.append(1)
for i in range(num_of_predictions - num_of_samples('src/airplane/images')):
    gt_airplane.append(0)
print(gt_airplane)

for i in range(num_of_samples('src/airplane/images')):
    gt_cannon.append(0)
for i in range(num_of_samples('src/cannon/images')):
    gt_cannon.append(1)
for i in range(num_of_samples('src/tank/images')):
    gt_cannon.append(0)
print(gt_cannon)

for i in range(num_of_samples('src/airplane/images')):
    gt_tank.append(0)
for i in range(num_of_samples('src/cannon/images')):
    gt_tank.append(0)
for i in range(num_of_samples('src/tank/images')):
    gt_tank.append(1)
print(gt_tank)


In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(num_of_predictions):
    if gt_airplane[i] == 1 and int_predicted_keys[i] == klases[0][0]:
        TP += 1
    elif gt_cannon[i] == 1 and int_predicted_keys[i] == klases[1][0]:
        TP += 1
    elif gt_tank[i] == 1 and int_predicted_keys[i] == klases[2][0]:
        TP += 1

for i in range(num_of_predictions):
    if gt_airplane[i] == 0 and int_predicted_keys[i] != klases[0][0]:
        TN += 1
    elif gt_cannon[i] == 0 and int_predicted_keys[i] != klases[1][0]:
        TN += 1
    elif gt_tank[i] == 0 and int_predicted_keys[i] != klases[2][0]:
        TN += 1
    
for i in range(num_of_predictions):
    if gt_airplane[i] == 0 and int_predicted_keys[i] == klases[0][0]:
        FP += 1
    elif gt_cannon[i] == 0 and int_predicted_keys[i] == klases[1][0]:
        FP += 1
    elif gt_tank[i] == 0 and int_predicted_keys[i] == klases[2][0]:
        FP += 1

for i in range(num_of_predictions):
    if gt_airplane[i] == 1 and int_predicted_keys[i] != klases[0][0]:
        FN += 1
    elif gt_cannon[i] == 1 and int_predicted_keys[i] != klases[1][0]:
        FN += 1
    elif gt_tank[i] == 1 and int_predicted_keys[i] != klases[2][0]:
        FN += 1

metrics = {}

metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
metrics['recall'] = TP / (TP + FN)
metrics['precision'] = TP / (TP + FP)
metrics['f1'] = 2 * (metrics['recall'] * metrics['precision']) / (metrics['recall'] + metrics['precision'])

print(metrics)